In [3]:
import openai
import json
import os
from dotenv import load_dotenv 
load_dotenv()
openai.api_key = os.getenv("openai.api_key")

In [2]:
messages = []

In [9]:
##### Reusable Code

def _openai() -> str:
    response = openai.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=messages,
    max_tokens=300,
    )

    return response.choices[0]

In [4]:
def request(image_url: str, prompt:str, type)-> str:
    if type == "E":
        messages.append({
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                "type": "image_url",
                "image_url": {
                    "url": image_url,
                },
                },
            ],
            }
            )
        _openai()
        

In [5]:
def extract(image_url:str) -> str:
    return request(
        image_url= image_url,
        prompt="Please analyze this picture and identify the social situation it depicts. Provide a detailed description of the scene, including the people present, their activities, and any notable features or characteristics. Your response should be formatted as a JSON object. The JSON should have a key named 'persons', which contains a list of individual person objects. Each person object should detail the person's activities and include descriptive elements such as clothing, posture, facial expressions, and any other observable features that would aid in recognizing them in a different picture."
    )

In [17]:
print(messages)

[{'role': 'user', 'content': [{'type': 'text', 'text': "Please analyze this picture and identify the social situation it depicts. Provide a detailed description of the scene, including the people present, their activities, and any notable features or characteristics. Your response should be formatted as a JSON object. The JSON should have a key named 'persons', which contains a list of individual person objects. Each person object should detail the person's activities and include descriptive elements such as clothing, posture, facial expressions, and any other observable features that would aid in recognizing them in a different picture."}, {'type': 'image_url', 'image_url': {'url': 'https://image.cnbcfm.com/api/v1/image/107295167-1693591029492-gettyimages-1522419916-a7409294.jpeg?v=1693839601&w=630&h=354&ffmt=webp&vtcrop=y'}}]}, {'role': 'user', 'content': [{'type': 'text', 'text': "Please analyze this picture and identify the social situation it depicts. Provide a detailed descriptio

In [15]:
def recognise(image_url:str, person_desciption:str)->str: 
    return request(
        image_url= image_url,
        prompt= f"Using the information from the previously analyzed picture, please identify a specific person using this description: {person_desciption} in a new image. Refer to the detailed descriptions from the 'persons' list in the provided serialized JSON object that can be safed in a JSON file. Focus on matching features such as clothing, hairstyle, facial expressions, posture, and any unique identifiers that were previously noted. Describe how each observable feature in the new image aligns with the information in the JSON object to confirm the identity of the person. If the person is present, provide additional information about that person."
    )

#####Application

In [16]:
social_situation: str = extract("https://image.cnbcfm.com/api/v1/image/107295167-1693591029492-gettyimages-1522419916-a7409294.jpeg?v=1693839601&w=630&h=354&ffmt=webp&vtcrop=y")

In [12]:
def choice_to_dict(choice_object):
    # This function should be adapted based on the structure of the Choice object
    # Example structure, adjust according to actual attributes
    return {
        'finish_reason': choice_object.finish_reason,
        'index': choice_object.index,
        'logprobs': choice_object.logprobs,
        'content': choice_object.message.content
        # Add other necessary fields here
    }

# Assuming _openai() returns an instance of the Choice class
data = _openai()

# Convert the Choice object to a dictionary
serializable_data = choice_to_dict(data)

file_path = 'Response_Vault.txt'

# Write the data to the file in JSON format
with open(file_path, 'w') as file:
    json.dump(serializable_data, file, indent=4)


TypeError: Object of type Choice is not JSON serializable

In [18]:
print(data)

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "persons": [\n    {\n      "description": "Woman on the left with curly hair",\n      "activities": "Laughing and leaning towards the center",\n      "clothing": "Wearing a green, long-sleeve top",\n      "posture": "Leaning forward slightly",\n      "facial_expression": "Joyful with eyes slightly closed",\n      "observable_features": [\n        "Curly hair",\n        "Green top",\n        "Left arm raised"\n      ]\n    },\n    {\n      "description": "Woman in the center holding a bowl",\n      "activities": "Smiling and holding a bowl of popcorn",\n      "clothing": "Wearing a plaid shirt",\n      "posture": "Sitting back and relaxed",\n      "facial_expression": "Happy with teeth visible",\n      "observable_features": [\n        "Blond hair",\n        "Plaid shirt",\n        "Holding a bowl"\n      ]\n    },\n    {\n      "description": "Woman on the right",\n      "activit

In [7]:

person_tobedetected = social_situation.persons[0]
recognision = recognise("...", person_tobedetected)

AttributeError: 'NoneType' object has no attribute 'persons'